In [ ]:
from dotenv import load_dotenv
import os
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain_groq import ChatGroq
from tavily import TavilyClient
# from langchain.vectorstores import Pinecone
from langchain_pinecone import PineconeVectorStore
# from langchain.embeddings import OpenAIEmbeddings
from pinecone import Pinecone
from typing import Annotated, Literal, TypedDict
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, START, StateGraph, MessagesState
load_dotenv()

GROQ_API_KEY = os.getenv('GROQ_API_KEY')
TAVILY_API_KEY = os.getenv('TAVILY_API_KEY')
PINE_API_KEY = os.getenv('PINE_API_KEY')

prompt = PromptTemplate(
    template="Question: {question}\n\nContext: {context}\n\nAnswer:",
    input_variables=["question", "context"]
)


pc = Pinecone(
    api_key=PINE_API_KEY,
)
index = pc.Index(
    'netsol-finance-asm4'
)
vector_store = PineconeVectorStore(index=index, embedding=HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2"))
model = ChatGroq(api_key=GROQ_API_KEY)
tavily = TavilyClient(api_key=TAVILY_API_KEY)



def category_decider(state: MessagesState) -> Literal["vector_store", "tavily"]:
    """
    A simple rule based checker to see in which category the user query lies
    """
    message = state['messages'][-1].content
    message = message.lower()

    if "netsol" in message:
        return "vector_store"
    else:
        return "tavily" 

    


def pinecone_retriever(state: MessagesState):

    query = state['messages'][-1].content
    retriever = vector_store.as_retriever()
    docs = retriever.invoke(query)

    return {
        "messages": [query, " ".join([doc.page_content for doc in docs])] 
        }

def tavily_retriever(state: MessagesState):
    query = state['messages'][-1].content
    results = tavily.search(query)['results']
    return {
        "messages":[query, " ".join([f"Content fetched from web with Title: {result['title']}\n and content: {result['content']}" for result in results])]
    }


def call_model(state: MessagesState):
    message = state['messages'][-1].content
    query = state['messages'][0].content

    messages = [
        (
            'system',
            "You are an LLM agent you will answer user queries based on the context provided."
        ),
        (
            'human',
            f'My Query is {query} Please answer Based on this context: {message}'
        )
    ]
    response = model.invoke(
        messages
    )
    return {"messages": [response.content]}

workflow = StateGraph(MessagesState)
workflow.add_node("agent", call_model)
workflow.add_node("tavily", tavily_retriever)
workflow.add_node("vector_store", pinecone_retriever)

workflow.add_conditional_edges(
    START, 
    category_decider
)
workflow.add_edge("tavily", "agent")
workflow.add_edge("vector_store", "agent")
workflow.add_edge("agent", END)

checkpointer = MemorySaver()
app = workflow.compile(checkpointer=checkpointer)

# Running the pipeline
query = "What awards has NETSOL Technologies won?"
# query = "Who is the CEO of Tesla ?"
final_response = app.invoke(
    {
        "messages": [HumanMessage(content=query)]
    },
    config={"configurable": {"thread_id": 40}}
)
print(final_response['messages'][-1].content)


Based on the provided context, the Chairman of NETSOL Technologies is not explicitly mentioned. However, the Annual General Meeting (AGM) is going to be held on October 18, 2024, and the Chairman is typically the person who chairs such meetings. If you need to know the current Chairman of NETSOL Technologies, I would recommend checking the company's website, their latest annual report, or contacting the company directly at corporate@netsolpk.com for the most accurate and up-to-date information.
